In [88]:
import os
import shutil
import random
from PIL import Image, ImageOps

In [96]:
images = "../datasets/images/"
bg_color = 'rgb(255, 255, 179)'

In [114]:
all_fonts = os.listdir(images)

if ".DS_Store" in all_fonts:
    del all_fonts[all_fonts.index(".DS_Store")]
    print('deleted DS_Store')



In [115]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_concat_hv(japs,eng,n,col):
    ctr = 0
    h = []
    for r in range(n-1):
#         print(ctr)

        h1 = ImageOps.colorize(japs[ctr], black = col, white = "white")
        ctr+=1
        if (r==(n//2)):
            for i in range(1,n):
                if i==(n//2):
                    h1=get_concat_h(h1, ImageOps.colorize(eng, black = "black", white = bg_color))
                    
                    continue
                h1=get_concat_h(h1,ImageOps.colorize(japs[ctr], black = col, white = "white"))
                ctr+=1
                
            h.append(h1)
            
            continue
        for i in range(1,n):
            h1=get_concat_h(h1,ImageOps.colorize(japs[ctr], black = col, white = "white"))
            ctr+=1
        h.append(h1)
                                
    h1=get_concat_h(ImageOps.colorize(japs[6], black = col, white = "white"),ImageOps.colorize(japs[5], black = col, white = "white"))
    h1=get_concat_h(h1,ImageOps.colorize(japs[7], black = col, white = "white"))
    
    h.append(h1)
    
#     h[-1]
    
    ans = h[0]
#     print(len(h))
    x=1
    while(x<len(h)):
        ans = get_concat_v(ans, h[x])
        x+=1                                
                    
    
    return ans.resize((256, 256), Image.ANTIALIAS)

In [116]:
t = "+ ".join(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ"))
english = t.split(" ") + list("abcdefghijklmnopqrstuvwxyz1234567890")


In [117]:
# creating training data
import numpy as np
from tqdm import tqdm
i = 0
while i<1000:
    col= f'rgb({random.randint(0,128)},{random.randint(0,128)},{random.randint(0,128)})'
    
    font_index = random.randint(0, len(all_fonts)-5)
    
    jap_alp = os.listdir(images + all_fonts[font_index])
    jap_alp.sort()
    jap_alp = jap_alp[63:]
    
    jap_alp_indexs = np.random.choice(list(range(0, len(jap_alp)-1)), 8,replace=True)
    
    eng_index = random.randint(0, len(english)-1)
    
    # read the images 
#     jap1 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[0]] ) 
#     jap2 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[1]] )
#     jap3 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[2]] )
    japs = [Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[i]] ) for i in range(8)]
    try:
        eng_output = Image.open(images + all_fonts[font_index] + '/' + english[eng_index] + ".png")
        eng_output = eng_output.resize((256,256), Image.ANTIALIAS)
        eng_output = ImageOps.colorize(eng_output, black = col, white = bg_color)
    except:
        continue
    
    eng = Image.open("./Gotham/" + english[eng_index] + ".png")
    
    X = get_concat_hv(japs, eng,3,col)
    
    # saving the training data
    X.save("A/train/" + str(i) + ".png")
    eng_output.save("B/train/" + str(i) + ".png")
    i +=1
    
    
    
    
print("Training data done")   
    
    

Training data done


In [118]:
# creating val data
i=0
while i<500:
    col= f'rgb({random.randint(0,128)},{random.randint(0,128)},{random.randint(0,128)})'
    
    font_index = random.randint(len(all_fonts)-6, len(all_fonts)-1)
    
    jap_alp = os.listdir(images + all_fonts[font_index])
    jap_alp.sort()
    jap_alp = jap_alp[63:]
    
    jap_alp_indexs = np.random.choice(list(range(0, len(jap_alp)-1)), 8,replace=True)
    
    eng_index = random.randint(0, len(english)-1)
    
    # read the images 
#     jap1 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[0]] ) 
#     jap2 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[1]] )
#     jap3 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[2]] )
    japs = [Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[i]] ) for i in range(8)]
    try:
        eng_output = Image.open(images + all_fonts[font_index] + '/' + english[eng_index] + ".png")
        eng_output = eng_output.resize((256,256), Image.ANTIALIAS)
        eng_output = ImageOps.colorize(eng_output, black = col, white = bg_color)

    except:
        continue
    
    eng = Image.open("./Gotham/" + english[eng_index] + ".png")
    
    X = get_concat_hv(japs, eng,3,col)

    
    
    # saving the training data
    X.save("A/val/" + str(i) + ".png")
    eng_output.save("B/val/" + str(i) + ".png")
    i+=1
    

print("Val data done")  
    
    
    
    
    

Val data done


In [119]:
# creating test data
i=0
while i<500:
    col= f'rgb({random.randint(0,128)},{random.randint(0,128)},{random.randint(0,128)})'
    
    
    font_index = random.randint(len(all_fonts)-6, len(all_fonts)-1)
    
    jap_alp = os.listdir(images + all_fonts[font_index])
    jap_alp.sort()
    jap_alp = jap_alp[63:]
    
    jap_alp_indexs = np.random.choice(list(range(0, len(jap_alp)-1)), 8,replace=True)
    
    eng_index = random.randint(0, len(english)-1)
    
    # read the images 
#     jap1 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[0]] ) 
#     jap2 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[1]] )
#     jap3 = Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[2]] )
    japs = [Image.open(images + all_fonts[font_index] + '/' + jap_alp[jap_alp_indexs[i]] ) for i in range(8)]
    try:
        eng_output = Image.open(images + all_fonts[font_index] + '/' + english[eng_index] + ".png")
        eng_output = eng_output.resize((256,256), Image.ANTIALIAS)
        eng_output = ImageOps.colorize(eng_output, black = col, white = bg_color)

    except:
        continue
    
    eng = Image.open("./Gotham/" + english[eng_index] + ".png")
    
    X = get_concat_hv(japs, eng,3,col)

    
    # saving the training data
    X.save("A/test/" + str(i) + ".png")
    eng_output.save("B/test/" + str(i) + ".png")
    i+=1

print("Test data done")  
    
    
    
    
    
    

Test data done
